# Setup

In [ ]:
import pandas as pd
from transformers import Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

In [ ]:
# control train dataset is in data/train/train.csv
# test dataset is in data/test/test.csv
# eval dataset is in data/eval/eval.csv
# real train dataset is in data/train_small/train_small.csv
# synthetic train dataset is in data/synthetic_data/synthetic_data.csv
# alternative big dataset tbd

# load all datasets
train = pd.read_csv('data/train/train.csv')
test = pd.read_csv('data/test/test.csv')
eval = pd.read_csv('data/eval/eval.csv')
real_train = pd.read_csv('data/train_small/train_small.csv')
synthetic_train = pd.read_csv('data/synthetic_data/synthetic_data.csv')

In [ ]:
# load wav2vec model
wav2vec = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-base-960h')

# load wav2vec feature extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-base-960h')

# load wav2vec processor
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')

# load wav2vec tokenizer
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained('facebook/wav2vec2-base-960h')

In [ ]:
# for all wav files in each dataset, get the embeddings for each dataset
#normalize the mean and variance of the embeddings for each dataset so that the mean is 0 and the variance is 1

# Crude Analysis

In [ ]:
# get the mean embedding for each dataset

In [ ]:
# compute the euclidean distance between the mean embedding of each dataset

In [ ]:
# compute the cosine similarity between the mean embedding of each dataset

# Information Theoretical Analysis

## Avoiding Estimating Probability Distributions

In [ ]:
# compute MMD between the embeddings of each dataset

In [ ]:
# Kolmogorov-Smirnov test between the embeddings of each dataset

In [ ]:
# Wasserstein distance between the embeddings of each dataset

## Estimating Probability Distributions

In [ ]:
# Jensen-Shannon divergence between the embeddings of each dataset

In [ ]:
# Bhattacharyya distance between the embeddings of each dataset